In [ ]:
# %% [markdown]
# # Results Summary
# 
# Final analysis and reporting of key findings.

# %%
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# %%
# Load all results
results_df = pd.read_csv('reports/results/model_results.csv')
promo_results = pd.read_csv('reports/results/promo_analysis_results.csv')

# %%
# Key metrics summary
print("=== Model Performance Summary ===")
print(f"Median Test MAE: {results_df['TestMAE'].median():.2f}")
print(f"Median Test R-squared: {results_df['TestR2'].median():.2f}")
print(f"Number of significant elasticities: {sum(results_df['ElasticityPval'] < 0.05)}/{len(results_df)}")

# %%
# Promotional impact summary
total_prior = promo_results['PriorRevenue'].sum()
total_forecast = promo_results['ForecastedRevenue'].sum()
lift = total_forecast - total_prior

print("\n=== Promotional Impact Summary ===")
print(f"Current Revenue: ${total_prior:,.2f}")
print(f"Forecasted Revenue: ${total_forecast:,.2f}")
print(f"Total Lift: ${lift:,.2f} ({lift/total_prior*100:.1f}%)")

# %%
# Create final report slides
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages('reports/final_report.pdf') as pdf:
    # Slide 1: Title
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis('off')
    ax.text(0.5, 0.7, 'Online Retail Pricing Analysis', 
            ha='center', va='center', fontsize=24, weight='bold')
    ax.text(0.5, 0.5, 'Price Elasticity Modeling & Promotional Impact', 
            ha='center', va='center', fontsize=18)
    pdf.savefig(fig)
    plt.close()
    
    # Slide 2: Model Performance
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.boxplot(data=results_df[['TrainMAE', 'TestMAE']], ax=ax)
    ax.set_title('Model Performance Across SKUs')
    ax.set_ylabel('Mean Absolute Error')
    pdf.savefig(fig)
    plt.close()
    
    # Slide 3: Elasticity Distribution
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(data=results_df, x='Elasticity', kde=True, ax=ax)
    ax.set_title('Distribution of Price Elasticity Estimates')
    ax.axvline(0, color='red', linestyle='--')
    pdf.savefig(fig)
    plt.close()
    
    # Slide 4: Revenue Impact
    fig, ax = plt.subplots(figsize=(10, 6))
    summary = pd.DataFrame({
        'Period': ['Current', 'Forecasted'],
        'Revenue': [total_prior, total_forecast]
    })
    sns.barplot(data=summary, x='Period', y='Revenue', 
                palette=["#135AE8D9", "#F8D700"], ax=ax)
    ax.set_title(f'Total Revenue Impact\nLift: ${lift:,.2f} ({lift/total_prior*100:.1f}%)')
    ax.set_ylabel('Revenue ($)')
    pdf.savefig(fig)
    plt.close()